In [ ]:
!pip install git+https://github.com/johmedr/dempy.git

In [ ]:
from dempy import *

import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

np.set_printoptions(linewidth=160, precision=2)

#### Generate some data
The model is: 
\begin{align} 
    y &= \theta_1 x \\ \dot{x} &= \theta_2 x + \theta_3 v 
\end{align}
where 
\begin{align} 
    \theta_1 = \begin{bmatrix} 
        0.1250 & 0.1633 \\
        0.1250 & 0.0676 \\ 
        0.1250 & -0.0676 \\ 
        0.1250 & -0.1633 
     \end{bmatrix} &&
     \theta_2 = \begin{bmatrix} 
         -0.25 & 1.00 \\
         -0.50 & -0.25 
     \end{bmatrix} && 
     \theta_3 = \begin{bmatrix} 
         1 \\ 0
     \end{bmatrix} 
\end{align}

We generate the data with $v = \exp\left(\frac{1}{4} (t - 12)^2\right)$. 

In [ ]:
theta1 = np.array([[0.125,  0.1633], 
                   [0.125,  0.0676], 
                   [0.125, -0.0676], 
                   [0.125, -0.1633]])
theta2 = np.array([[-0.25,  1.00],
                   [-0.50, -0.25]])
theta3 = np.array([[1.], [0.]])

# Prior expectation on the parameters
pE  = np.concatenate([theta1.reshape((-1,)), theta2.reshape((-1,)), theta3.reshape((-1,))])

nps = (theta1.size,theta2.size,theta3.size)

def g_(x, v, p): 
    return p[:nps[0]].reshape(theta1.shape) @ x

def f_(x, v, p): 
    return p[nps[0]:nps[0] + nps[1]].reshape(theta2.shape) @ x + p[-nps[2]:].reshape(theta3.shape) @ v

In [ ]:
models = [
    GaussianModel(
        fsymb=f_, gsymb=g_, 
        n=2, sv=1./2,sw=1./2,
        V=np.array([np.exp(8.)]), 
        W=np.array([np.exp(8.)]), 
        pE=pE, pC=np.ones_like(pE) * np.exp(-64), xP=np.array([12])
    ), 
    GaussianModel(l=1, V=np.array([np.exp(32.)]))
]
genmodel = HierarchicalGaussianModel(*models)

Run the model to generate some data.

In [ ]:
nT = 32
t  = np.arange(1, nT+1)  

# Input vector
u  = (np.exp(-(t - 12)**2/4))[:, None]

# Generate the data
gen = DEMInversion(genmodel, states_embedding_order=5).generate(nT, u)

plot_dem_generate(genmodel, gen);

# Store y as an "observed" trajectory
y   = gen.v[:,0,:4]

In [ ]:
decmodel      = genmodel.copy()

# Change the prior precision of the higher level causes (<-> assume unknown input)
decmodel[1].V = np.ones((1,1))

In [ ]:
# Run inversion
deminv  = DEMInversion(decmodel, states_embedding_order=4)
results = deminv.run(y, nD=1, nE=1, nM=1, K=1, td=1)

In [ ]:
plot_dem_states(genmodel, results, gen);

In [ ]:
nps   = (theta1.size,theta2.size,theta3.size)
ip    = [0, 10] 
P     = pE.copy()
P[0]  = 0.6
P[10] = -0.2
pC    = np.zeros_like(pE, dtype='d')
pC[ip]= np.exp(4)
# cC    = np.empty_like(pE , dtype=object)
# cC[0] = 'positive'
# cC[10] = 'negative'

models = [
    GaussianModel(
        g=g_, f=f_,
        n=2, sv=1., sw=1.,
#         V=np.array([np.exp(8.)]), 
#         W=np.array([np.exp(16.)]), 
        Q=[np.eye(4)], R=[np.eye(2)],
        hE=np.array([6.]), hC=np.array([[1/4.]]),
        gE=np.array([6.]), gC=np.array([[1/4.]]), 
        pE=P.copy(), pC=pC, #constraints=cC
    ), 
    GaussianModel(l=1, V=np.array([np.exp(0)]))
]
decdualmodel = HierarchicalGaussianModel(*models)

In [ ]:
deminv  = DEMInversion(decdualmodel, states_embedding_order=6)
deminv.logger.setLevel('ERROR')
results = deminv.run(y.copy(), u.copy(), nD=1, nE=128, nM=8, K=1., Emin=64, Mmin=8, tol=np.finfo(np.float64).eps)
px.line(y=[results.F]).update_layout(template='simple_white', height=400, width=800)

In [ ]:
pE[0], results.qP.P[0], results.qP.V[0], pE[10],  results.qP.P[10], 

In [ ]:
plot_dem_states(decdualmodel, results, gen);